In [2]:
import os, duckdb

DATA_DIR = r"C:\Users\User\Desktop\ML\Project\solar-potential-analysis-github-setup\cleaned_datasets"
PATTERN  = os.path.join(DATA_DIR, "*.csv")
con = duckdb.connect()  # in-memory is fine for EDA

# Read all CSVs leniently and align columns by name
con.execute(f"""
CREATE OR REPLACE VIEW all_raw AS
SELECT *
FROM read_csv_auto(
  '{PATTERN}',
  filename=true,
  union_by_name=true,
  files_to_sniff=-1,
  sample_size=-1,
  strict_mode=false,
  ignore_errors=true,
  max_line_size=10000000
);
""")

# Force a stable schema
con.execute("""
CREATE OR REPLACE VIEW all_data AS
SELECT
  City::VARCHAR                                   AS City,
  TRY_CAST(Surface_area AS DOUBLE)                AS Surface_area,
  TRY_CAST(Potential_installable_area AS DOUBLE)  AS Potential_installable_area,
  TRY_CAST(Peak_installable_capacity AS DOUBLE)   AS Peak_installable_capacity,
  TRY_CAST(Energy_potential_per_year AS DOUBLE)   AS Energy_potential_per_year,
  NULLIF(TRIM(CAST(Assumed_building_type AS VARCHAR)), '') AS Assumed_building_type,
  TRY_CAST(Estimated_tilt AS DOUBLE)              AS Estimated_tilt,
  TRY_CAST(Estimated_building_height AS DOUBLE)   AS Estimated_building_height,
  TRY_CAST(Estimated_capacity_factor AS DOUBLE)   AS Estimated_capacity_factor,
  filename                                        AS source_file
FROM all_raw;
""")

print(con.execute("DESCRIBE SELECT * FROM all_data").df())

                  column_name column_type null   key default extra
0                        City     VARCHAR  YES  None    None  None
1                Surface_area      DOUBLE  YES  None    None  None
2  Potential_installable_area      DOUBLE  YES  None    None  None
3   Peak_installable_capacity      DOUBLE  YES  None    None  None
4   Energy_potential_per_year      DOUBLE  YES  None    None  None
5       Assumed_building_type     VARCHAR  YES  None    None  None
6              Estimated_tilt      DOUBLE  YES  None    None  None
7   Estimated_building_height      DOUBLE  YES  None    None  None
8   Estimated_capacity_factor      DOUBLE  YES  None    None  None
9                 source_file     VARCHAR  YES  None    None  None


In [3]:
# Per-file row counts
files_df = con.execute("""
SELECT source_file AS file, COUNT(*) AS n_rows
FROM all_data
GROUP BY file ORDER BY n_rows DESC
""").df()

# Overall overview
overview_df = con.execute("""
SELECT 
  COUNT(*) AS n_rows,
  COUNT(DISTINCT City) AS n_cities,
  COUNT(DISTINCT Assumed_building_type) AS n_btypes
FROM all_data
""").df()

# Per-city size + target mean
city_size_df = con.execute("""
SELECT City, COUNT(*) AS n, AVG(Energy_potential_per_year) AS mean_energy
FROM all_data
GROUP BY City ORDER BY n DESC
""").df()

# Global missingness
missing_global = con.execute("""
SELECT
  SUM(Surface_area IS NULL)                 AS na_surface_area,
  SUM(Potential_installable_area IS NULL)   AS na_pia,
  SUM(Peak_installable_capacity IS NULL)    AS na_pic,
  SUM(Energy_potential_per_year IS NULL)    AS na_energy,
  SUM(Estimated_tilt IS NULL)               AS na_tilt,
  SUM(Estimated_building_height IS NULL)    AS na_height,
  SUM(Estimated_capacity_factor IS NULL)    AS na_cf,
  SUM(Assumed_building_type IS NULL)        AS na_btype
FROM all_data
""").df()

# Top 10 cities with highest target missing rate
missing_city = con.execute("""
SELECT City,
       AVG(Energy_potential_per_year IS NULL)::DOUBLE AS energy_null_rate
FROM all_data
GROUP BY City
ORDER BY energy_null_rate DESC
LIMIT 10
""").df()

# Approx quantiles (outlier scan)
quantiles = con.execute("""
WITH base AS (SELECT * FROM all_data)
SELECT 'Energy_potential_per_year' AS var,
  APPROX_QUANTILE(Energy_potential_per_year, 0.01) AS p01,
  APPROX_QUANTILE(Energy_potential_per_year, 0.50) AS p50,
  APPROX_QUANTILE(Energy_potential_per_year, 0.99) AS p99
UNION ALL
SELECT 'Surface_area',
  APPROX_QUANTILE(Surface_area, 0.01), APPROX_QUANTILE(Surface_area, 0.50), APPROX_QUANTILE(Surface_area, 0.99) FROM base
UNION ALL
SELECT 'Potential_installable_area',
  APPROX_QUANTILE(Potential_installable_area, 0.01), APPROX_QUANTILE(Potential_installable_area, 0.50), APPROX_QUANTILE(Potential_installable_area, 0.99) FROM base
UNION ALL
SELECT 'Peak_installable_capacity',
  APPROX_QUANTILE(Peak_installable_capacity, 0.01), APPROX_QUANTILE(Peak_installable_capacity, 0.50), APPROX_QUANTILE(Peak_installable_capacity, 0.99) FROM base
UNION ALL
SELECT 'Estimated_tilt',
  APPROX_QUANTILE(Estimated_tilt, 0.01), APPROX_QUANTILE(Estimated_tilt, 0.50), APPROX_QUANTILE(Estimated_tilt, 0.99) FROM base
UNION ALL
SELECT 'Estimated_building_height',
  APPROX_QUANTILE(Estimated_building_height, 0.01), APPROX_QUANTILE(Estimated_building_height, 0.50), APPROX_QUANTILE(Estimated_building_height, 0.99) FROM base
UNION ALL
SELECT 'Estimated_capacity_factor',
  APPROX_QUANTILE(Estimated_capacity_factor, 0.01), APPROX_QUANTILE(Estimated_capacity_factor, 0.50), APPROX_QUANTILE(Estimated_capacity_factor, 0.99) FROM base
""").df()

# Leakage check: is energy ≈ capacity * cf * constant?
ratio_q = con.execute("""
SELECT 
  APPROX_QUANTILE(
    Energy_potential_per_year / NULLIF(Peak_installable_capacity * Estimated_capacity_factor, 0), 0.5
  ) AS ratio_median,
  APPROX_QUANTILE(
    Energy_potential_per_year / NULLIF(Peak_installable_capacity * Estimated_capacity_factor, 0), 0.1
  ) AS ratio_p10,
  APPROX_QUANTILE(
    Energy_potential_per_year / NULLIF(Peak_installable_capacity * Estimated_capacity_factor, 0), 0.9
  ) AS ratio_p90
FROM all_data
""").df()

# 1% stratified-ish sample for correlations
corr_df = con.execute("""
WITH S AS (
  SELECT * FROM all_data USING SAMPLE 1% (BERNOULLI)
)
SELECT Surface_area, Potential_installable_area, Peak_installable_capacity,
       Energy_potential_per_year, Estimated_tilt, Estimated_building_height,
       Estimated_capacity_factor
FROM S
""").df()

corr = corr_df.corr(method="spearman").round(3)


BinderException: Binder Error: No function matches the given name and argument types 'avg(BOOLEAN)'. You might need to add explicit type casts.
	Candidate functions:
	avg(DECIMAL) -> DECIMAL
	avg(SMALLINT) -> DOUBLE
	avg(INTEGER) -> DOUBLE
	avg(BIGINT) -> DOUBLE
	avg(HUGEINT) -> DOUBLE
	avg(INTERVAL) -> INTERVAL
	avg(DOUBLE) -> DOUBLE
	avg(TIMESTAMP) -> TIMESTAMP
	avg(TIMESTAMP WITH TIME ZONE) -> TIMESTAMP WITH TIME ZONE
	avg(TIME) -> TIME
	avg(TIME WITH TIME ZONE) -> TIME WITH TIME ZONE


LINE 3:        AVG(Energy_potential_per_year IS NULL)::DOUBLE AS energy_nu...
               ^